In [67]:
import csv
import string
import re
from sklearn import svm
import sklearn
import gzip
import random
import itertools
from collections import defaultdict
import numpy as np
import datetime

#from sklearn.metrics import jaccard_similarity_score
#from scipy.sparse import csc_matrix

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm

In [68]:
#players = []
#heroNames = []
#matches = []

data = []


with open('match.csv') as csvfile:
    keys = list(csvfile.readline().split(',')) #note: last element has a newline
    spamreader = csv.reader(csvfile, delimiter=',')
    desiredKeys = ['match_id', 'duration', 'radiant_win']
    types = [int, int, lambda x: x == "True"]
    desiredKeysIndices = [keys.index(x) for x in desiredKeys]
    for row in spamreader:
        datum = {}
        for i in range(len(desiredKeys)):
            datum[desiredKeys[i]] = types[i](row[desiredKeysIndices[i]])
        datum['players'] = []
        data.append(datum)


with open('players.csv') as csvfile:
    keys = list(csvfile.readline().split(','))
    spamreader = csv.reader(csvfile, delimiter=',')
    desiredKeys = ['hero_id'] # just heroId for now
    types = [int]
    desiredKeysIndices = [keys.index(x) for x in desiredKeys]
    for row in spamreader:
        datum = {}
        for i in range(len(desiredKeys)):
            datum[desiredKeys[i]] = types[i](row[desiredKeysIndices[i]])
        data[int(row[0])]['players'].append(datum)
        


"""
with open('hero_names.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        heroNames.append([row[1], row[2]])
"""

"\nwith open('hero_names.csv', newline='') as csvfile:\n    spamreader = csv.reader(csvfile, delimiter=',')\n    for row in spamreader:\n        heroNames.append([row[1], row[2]])\n"

In [69]:
random.shuffle(data)
train = data[:40000]
val = data[40000:]

In [70]:
def accuracy(val_lbls, pred):
    numright = 0
    for i in range(len(val_lbls)):
        if val_lbls[i] == pred[i]:
            numright += 1
    return float(numright) / len(val_lbls)

def classify(tf, tl, vf, vl):
    classifiers = [
    #KNeighborsClassifier(3),
    svm.LinearSVC(C=1),
    #svm.LinearSVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=4),
    RandomForestClassifier(max_depth=4, n_estimators=50, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    #QuadraticDiscriminantAnalysis()
    ]
    
    for clf in classifiers:
        clf.fit(tf, tl)
        val_predictions = clf.predict(vf)
        print accuracy(vl, val_predictions)


def baseline(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    def feature(d):
        feat = [0]*226
        for p in d['players'][:5]:
            feat[p['hero_id']] = 1
        for p in d['players'][5:]:
            feat[p['hero_id'] + 113] = 1
        return feat

    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [71]:
baseline(train, val)

0.5962
0.5222
0.517
0.6006
0.5837
0.5825
